In [ ]:
import pickle
import sklearn
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Read in the trained model and the data matrix

model = pickle.load(open('model_pkl_file','rb'))
dat = pickle.load(open('data_pkl_file','rb'))

In [ ]:
# Determine number of features, as well as their min and max

X = dat['test_x']
rf = model['modelobj']
feat_max = np.amax(X,axis=0)
feat_min = np.amin(X,axis=0)
n_feat = len(feat_min)

In [ ]:
# Generate random samples, run the trained model on it, and join

random_X = np.array([[random.uniform(feat_min[i],feat_max[i]) for i in range(n_feat)] for _ in range(100000)])
random_y = rf.predict_proba(random_X)[:,1]
mc_mat = np.c_[random_X,random_y]

In [ ]:
# Perform the Monte Carlo integration

risk_score_curves = []

for i in range(len(feat_max)):
    ssize = (feat_max[i] - feat_min[i])/50.
    probs = []
    for j in range(50):
        sub_mat = mc_mat[ (mc_mat[:,i]>feat_min[i]+j*ssize) & (mc_mat[:,i]<feat_min[i]+(j+1)*ssize) , : ]
        probs.append([np.sum(sub_mat[:,-1])/len(sub_mat[:,-1]),np.std(sub_mat[:,-1])])
    risk_score_curves.append(probs)

In [ ]:
# Remove features that the method doesn't currently work with: categorical variables and features with only one feature value

nonanarr = []

for i in range(len(feat_max)):
    if math.isnan(np.array(risk_score_curves)[i,0,0])==True: 
        #print(model['feature_importances_names'][i])
        #print(feat_min[i],feat_max[i])
    if math.isnan(np.array(risk_score_curves)[i,0,0])==False: nonanarr.append(i)

In [ ]:
# Perform edge detection, print and plot transitions

for xx in nonanarr:
    curve = np.array(risk_score_curves)[xx,:,:]
    vals = curve[:,0]
    sigmas = curve[:,1]
    subarr = []
    subarr.append(vals[0])
    transitions = []
    
    for i in range(49):
        mean = np.mean(subarr)
        subarr.append(vals[i+1])
        std = np.std(subarr)
        
        #if abs(vals[i+1]-mean) > 2.*std:
        if abs(vals[i+1]-mean) > 0.1 * sigmas[i+1]:
            print("!!!", i, vals[i+1]-mean)
            subarr = []
            subarr.append(vals[i+1])
            transitions.append([i,vals[i+1]-mean])
            
    plt.figure(figsize=(8,6))
    plt.title(model['feature_importances_names'][xx])
    ssize = (feat_max[xx]-feat_min[xx])/50.
    feat_range = np.arange(feat_min[xx]+0.5*ssize, feat_max[xx],ssize)
    plt.errorbar(feat_range, vals, sigmas)
    
    for i in range(len(transitions)):
        xpos = transitions[i][0]
        plt.arrow((feat_range[xpos]+feat_range[xpos+1])/2.,(vals[xpos]+vals[xpos+1])/2.,0,transitions[i][1],head_length=0.002,head_width=ssize/1.2,width=ssize/10.,color='red')
    
    #plt.savefig(model['feature_importances_names'][xx]+'.png')
    plt.show()
    plt.clf()

In [ ]:
# The old definition for directed importances

master = np.array(risk_score_curves)
directed_importances = []

for i in range(n_feat):
    directed_importances.append(master[i,-1,0]-master[i,0,0])
    
print(np.c_[model['feature_importances_names'],directed_importances])